##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 영화 리뷰를 사용한 텍스트 분류

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북에 다운로드</a>
  </td>
</table>

본 튜토리얼은 영화 리뷰(review) 텍스트를 긍정(positive) 또는 부정(negative)으로 분류하는 방법에 대해 소개합니다. 이 예제는 클래스(class)가 두 개인 이진 분류(binary classification) 문제의 일종입니다. 이진 분류는 중요하고 흔히 볼 수 있는 머신러닝 문제들 중 하나입니다.

여기에서는 [인터넷 영화 데이터베이스](https://www.imdb.com/)(Internet Movie Database)에서 수집한 50,000개의 영화 리뷰 텍스트를 담은 [IMDB 데이터셋](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb)을 사용하겠습니다. 25,000개 리뷰는 훈련용으로, 25,000개는 테스트용으로 나뉘어져 있습니다. 훈련 세트와 테스트 세트의 클래스는 *균형*이 잡혀 있습니다. 즉 긍정적인 리뷰와 부정적인 리뷰의 개수가 동일합니다.

이 노트북은 모델을 만들고 훈련하기 위해 텐서플로의 고수준 파이썬 API인 [tf.keras](https://www.tensorflow.org/guide/keras)를 사용합니다. `tf.keras`를 사용한 고급 텍스트 분류 튜토리얼은 [MLCC 텍스트 분류 가이드](https://developers.google.com/machine-learning/guides/text-classification/)를 참고하세요.

## 준비하기

In [0]:
import tensorflow as tf

In [0]:
from tensorflow import keras

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import numpy as np

print(tf.__version__)

<a id="download"></a>

## IMDB 데이터셋 다운로드

IMDB 영화 리뷰 데이터셋은 `tfds` 형태로 제공됩니다. 이 데이터셋은 이미 전처리를 거쳐 각 리뷰(단어의 배열)가 정수의 배열로 변환되었습니다. 이 때 각 숫자는 말뭉치에 있는 단어를 나타냅니다.

다음 코드는 IMDB 데이터셋을 내려받습니다 (또는 이전에 이미 내려받은 적이 있다면 캐시된 복사본을 사용합니다):

텍스트를 인코딩하려면 [튜토리얼 문서](../load_data/text.ipynb)를 참조하세요.

In [0]:
(train_data, test_data), info = tfds.load(
    # 사전 인코딩된 버전(단어 8k) 사용.
    'imdb_reviews/subwords8k', 
    # 훈련/테스트 데이터셋을 튜플로 반환.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # 사전 대신 데이터 집합에서 쌍(예: 레이블)을 반환.
    as_supervised=True,
    # info 구조도 반납. 
    with_info=True)

<a id="encoder"></a>

## 인코더 시험해보기

 데이터셋 `info`는 텍스트 인코더를 포함하고 있습니다 (`tfds.features.text.SubwordTextEncoder`).

In [0]:
encoder = info.features['text'].encoder

In [0]:
print ('말뭉치 크기: {}'.format(encoder.vocab_size))

이 텍스트 인코더는 문자열을 인코딩합니다 (인코딩된 문자열은 디코딩을 거쳐 원형으로 복원될 수 있습니다):

In [0]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print ('인코딩된 문자열: {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('원래의 문자열: "{}"'.format(original_string))

assert original_string == sample_string

인코더는 주어진 단어가 사전에 없을 경우 그 단어를 더 작은 단위의 단어나 문자로 분해하여 인코딩합니다. 따라서 입력 문자열이 데이터셋과 비슷할수록 인코딩된 표현의 길이는 더 짧을 것입니다.

In [0]:
for ts in encoded_string:
  print ('{} ----> {}'.format(ts, encoder.decode([ts])))

## 데이터 둘러보기

데이터의 형식을 이해하는 시간을 가져봅시다. 데이터셋은 이미 전처리가 되어있습니다: 각 예는 영화 리뷰의 단어를 나타내는 정수들의 배열입니다. 

리뷰 텍스트는 정수 배열들로 변환되었는데, 이 때 각 정수는 사전의 특정 단어, 혹은 단어보다 작은 단위의 말을 나타냅니다. 

각 레이블은 0 또는 1의 정수 값이며, 여기서 0은 부정적인 리뷰, 1은 긍정적인 리뷰를 나타냅니다.

첫 번째 리뷰는 다음과 같습니다:

In [0]:
for train_example, train_label in train_data.take(1):
  print('인코딩된 문자:', train_example[:10].numpy())
  print('레이블:', train_label.numpy())

`info` 구조는 인코더/디코더를 포함하고 있습니다. 인코더를 사용하여 원본 텍스트를 복구할 수 있습니다.:

In [0]:
encoder.decode(train_example)

## 훈련 데이터 준비하기

모델 훈련을 위한 배치 데이터를 만들어봅시다. 각 리뷰의 길이가 모두 다르므로, 길이를 통일하기 위해 `padded_batch`를 사용하여 짧은 리뷰는 0으로 패딩합니다:

In [0]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32))

test_batches = (
    test_data
    .padded_batch(32))

각 배치의 모양은 `(batch_size, sequence_length)` 입니다. 패딩은 동적으로 이뤄지기 때문에 각 배치의 길이가 서로 다릅니다:

In [0]:
for example_batch, label_batch in train_batches.take(2):
  print("배치 모양:", example_batch.shape)
  print("레이블 모양:", label_batch.shape)
  

## 모델 만들기

신경망은 층을 쌓아서 만듭니다. 이를 위해서는 두 가지 주요한 구조적 결정을 내려야합니다.:

* 모델에서 얼마나 많은 층을 사용할 것인가?
* 각 층에서 얼마나 많은 *은닉 유닛(hidden unit)*을 사용할 것인가?

이 예제의 입력 데이터는 단어 인덱스의 배열입니다. 예측할 레이블은 0 또는 1입니다. 이 문제에 대한 "연속적인 단어 모음 (Continuous bag of words)" 스타일 모델을 구축해 봅시다:

경고: 이 모델은 마스킹(masking)을 사용하지 않기 때문에 제로 패딩이 입력의 일부로 사용되므로 패딩 길이가 출력에 영향을 줄 수 있습니다. 이 문제점을 해결하기 위해서는 [마스킹 및 패딩 가이드](../../guide/keras/masking_and_padding.ipynb)을 참고하세요.

In [0]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1)])

model.summary()

층을 순서대로 쌓아 분류기(classifier)를 만듭니다:

1. 첫 번째 층은 `Embedding` 층입니다. 이 층은 정수로 인코딩된 단어를 입력으로 받고 각 단어 인덱스에 대한 임베딩 벡터를 찾습니다. 이 벡터는 모델이 훈련되면서 학습됩니다. 이 벡터는 출력 배열에 새로운 차원으로 추가됩니다. 최종 차원은 `(batch, sequence, embedding)`입니다.  *임베딩에 대해서 더 알고 싶다면, [임베딩 튜토리얼](../text/word_embeddings.ipynb)를 참조하세요.*
2. 다음, `GlobalAveragePooling1D` 층은 `sequence` 차원에 대해 평균을 계산하여 각 샘플에 대해 고정된 길이의 출력 벡터를 반환합니다. 이는 길이가 다른 입력을 다루는 가장 간단한 방법입니다.
3. 이러한 고정 길이의 출력 벡터는 16개의 은닉 유닛을 가진 완전히 연결된(fully connected) `Dense` 층을 거칩니다.
4. 마지막 층은 하나의 출력 노드(node)를 가진 완전 연결 층입니다. 여기서는 수치적 안정성을 위해 로짓을 출력하는 기본 선형(linear) 활성화 함수를 사용합니다. 다른 옵션은 확률 또는 신뢰도를 나타내기 위해 0에서 1 사이의 실수값을 반환하는 *시그모이드* 활성화 함수를 사용하는 것입니다.

### 은닉 유닛

위 모델은 입출력 사이에 두 개의 중간 또는 "은닉" 층이 있습니다. 출력(유닛, 노드, 또는 뉴런)의 갯수는 층을 나타내는 표현 공간(representation space)의 차원, 즉 내부 표현을 학습할 때 허용되는 네트워크 자유도의 양입니다.

모델에 많은 은닉 유닛(고차원의 표현 공간)과 층이 있다면 네트워크는 더 복잡한 표현을 학습할 수 있게 됩니다. 하지만 계산 비용이 증가하고 원치않는 패턴을 학습할 수도 있다는 부작용이 존재합니다. 이는 훈련 데이터에 대한 모델의 성능은 향상시키지만 테스트 데이터에 대한 성능은 저하시키는 역효과를 일으킬 수도 있습니다. 이를 *과대적합*(overfitting)이라고 부르는데, 이 현상에 대해서는 나중에 알아보도록 하겠습니다.

### 손실 함수와 옵티마이저

모델 훈련을 위해서는 손실 함수(loss function)와 옵티마이저(optimizer)가 필요합니다. 이 예제는 이진 분류 문제이고 모델은 로짓(logit, 선형 활성화 함수를 거친 단일 층)을 출력하므로 `binary_crossentropy` 손실 함수를 사용합니다.

다른 손실 함수를 선택할 수 없는 것은 아닙니다. 예를 들어 `mean_squared_error`를 선택할 수 있습니다. 하지만 일반적으로 `binary_crossentropy`가 확률을 다루는데 더 적합합니다. 이 함수는 확률 분포 간의 거리를 측정합니다. 여기에서는 정답인 타깃 분포와 예측 분포 사이의 거리입니다.

나중에 회귀(regression) 문제(예를 들어 주택 가격을 예측하는 문제)에 대해 살펴 볼 때 평균 제곱 오차(mean squared error) 손실 함수를 어떻게 사용하는지 알아 보겠습니다.

이제 모델이 사용할 옵티마이저와 손실 함수를 설정해 보겠습니다:

In [0]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 모델 훈련

`Dataset`을 fit 함수의 인자로 전달하여 모델을 훈련합니다. 에폭 수도 설정합시다.

In [0]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

## 모델 평가하기

모델의 성능을 확인해봅시다. 두 개의 값이 반환되는데, 이는 각각 손실(오차를 나타내는 숫자이므로 낮을수록 좋습니다)과 정확도입니다.

In [0]:
loss, accuracy = model.evaluate(test_batches)

print("손실: ", loss)
print("정확도: ", accuracy)

이 예제는 상당히 단순한 접근법으로 약 87%의 정확도를 달성했습니다. 고급 방법을 사용한 모델은 95%에 가까운 정확도를 얻습니다.

## 시간에 따른 정확도 및 손실 그래프 그리기

`model.fit()`은 `History` 객체를 반환합니다. 여기에는 훈련하는 동안 일어난 모든 정보가 담긴 딕셔너리(dictionary)가 들어 있습니다:

In [0]:
history_dict = history.history
history_dict.keys()

네 개의 항목이 있습니다. 훈련과 검증 단계에서 모니터링하는 지표들입니다. 훈련 손실과 검증 손실을 그래프로 그려 보고, 훈련 정확도와 검증 정확도도 그래프로 그려서 비교해 보겠습니다:

In [0]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# # "bo"는 "파란색 점"입니다
plt.plot(epochs, loss, 'bo', label='Training loss')
## b는 "파란 실선"입니다
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # 그림을 초기화합니다.

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

이 그래프에서 점선은 훈련 손실과 훈련 정확도를 나타냅니다. 실선은 검증 손실과 검증 정확도입니다.

훈련 손실은 에포크마다 *감소*하고 훈련 정확도는 *증가*한다는 것을 주목하세요. 경사 하강법 최적화를 사용할 때 볼 수 있는 현상입니다. 매 반복마다 최적화 대상의 값을 최소화합니다.

하지만 검증 손실과 검증 정확도에서는 그렇지 못합니다. 약 20번째 에포크 이후가 최적점인 것 같습니다. 이는 과대적합 때문입니다. 이전에 본 적 없는 데이터보다 훈련 데이터에서 더 잘 동작합니다. 이 지점부터는 모델이 과도하게 최적화되어 테스트 데이터에서 *일반화*되기 어려운 훈련 데이터의 특정 표현을 학습합니다.

여기에서는 과대적합을 막기 위해 단순히 20번째 에포크 근처에서 훈련을 멈출 수 있습니다. 나중에 콜백(callback)을 사용하여 자동으로 이렇게 하는 방법을 배워 보겠습니다.